In [49]:
#|default_exp losses
#|export
import torch
import numpy as np
import pandas as pd
from tsai.basics import *


# Custom losses

In [50]:
#|export
class Loss():
    def __init__(self, ranges, weights, device):
        self.device = device
        self.ranges = torch.Tensor(ranges).to(device)
        self.weights = torch.Tensor(weights).to(device)
    
    def weighted_loss_tensor(self, target):
        batch, variables, horizon = target.shape # Shape (32, 4, 6)
        variable, range, interval = self.ranges.shape # Shape (4, 4, 2)

        target_shaped = torch.reshape(target, (batch, variables, 1, horizon)) # Shape (32, 4, 6) -> (32, 4, 1, 6)
        ranges_shaped = torch.reshape(self.ranges, (variable, range, 1, interval)) # Shape (4, 4, 2) -> (4, 4, 1, 2)

        weights_tensor = (1 + ((ranges_shaped[..., 0] <= target_shaped) * (target_shaped <= ranges_shaped[...,1]))).float().to(self.device)
        
        return torch.einsum('r,bvrh->bvh', self.weights, weights_tensor)

In [46]:
#|export
class wMSELoss(nn.Module, Loss):
    def __init__(self, ranges, weights, device):
        nn.Module.__init__(self)
        Loss.__init__(self ,ranges, weights, device)

    
    def forward(self, y_pred, y_true):
        return torch.mean(self.weighted_loss_tensor(y_true) * (y_pred - y_true) ** 2).cpu()

In [47]:
#|export
class wMAELoss(nn.Module, Loss):
    def __init__(self, ranges, weights, device):
        nn.Module.__init__(self)
        Loss.__init__(self ,ranges, weights, device)

    
    def forward(self, y_pred, y_true):
        return torch.mean(self.weighted_loss_tensor(y_true) * torch.abs(y_pred - y_true)).cpu()

In [56]:
#|export

class LossMetrics():
    def __init__(self, loss:Loss, ranges, weights, device):
        self.ranges = ranges
        self.weights = weights
        self.loss = loss
        self.device = device
    
    def loss_low(self, input, target):
        return self.loss(self.ranges, weights=[self.weights[0],0,0,0], device=self.device).forward(input, target)
    def loss_moderate(self, input, target):
        return self.loss(self.ranges, weights=[0, self.weights[1],0,0], device=self.device).forward(input, target)
    def loss_elevated(self, input, target):
        return self.loss(self.ranges, weights=[0,0,self.weights[2],0], device=self.device).forward(input, target)
    def loss_high(self, input, target):
        return self.loss(self.ranges, weights=[0,0,0,self.weights[3]], device=self.device).forward(input, target)

    def metrics(self):
        return [self.mse_loss_low, self.mse_loss_moderate, self.mse_loss_elevated, self.mse_loss_high]

In [48]:
weights = [0.1, 0.2, 0.3, 0.4]
ranges = [[[0, 1], [0, 1], [0, 1], [0, 1]],
            [[0, 1], [0, 1], [0, 1], [0, 1]],
            [[0, 1], [0, 1], [0, 1], [0, 1]],
            [[0, 1], [0, 1], [0, 1], [0, 1]]]

loss = wMAELoss(ranges, weights, torch.device('cuda:0'))

input = torch.rand(32, 4, 6).to('cuda:0')
target = torch.rand(32, 4, 6).to('cuda:0')

print(loss.forward(input, target))

tensor(0.6575)
